In [ ]:
from SoccerNet.Downloader import SoccerNetDownloader
from SoccerNet.utils import getListGames

import pandas as pd

In [ ]:
# List all games
all_games = getListGames(split=["train", "valid", "test", "challenge"])
print(f"Total games: {len(all_games)}")

In [ ]:
df = pd.DataFrame(all_games, columns=["Games"])
df.iloc[1]

In [ ]:
df["Games"].iloc[1].split("/")

In [ ]:


# Regex to extract all info
pattern = r'^(?P<league>[^\\]+)\\(?P<season>[^\\]+)\\(?P<date>\d{4}-\d{2}-\d{2})\s*-\s*(?P<time>\d{2}-\d{2})\s+(?P<home_team>.+?)\s+(?P<home_score>\d+)\s*-\s*(?P<away_score>\d+)\s+(?P<away_team>.+)$'

parsed = df["Games"].str.extract(pattern)

# Reorder columns
parsed = parsed[["league","season","date","time","home_team","home_score","away_score","away_team"]]

parsed


In [ ]:
parsed.isna().sum()

In [ ]:
import pandas as pd

# Example parsed DataFrame
# parsed = pd.DataFrame(...)

def interactive_search(df):
    print("Enter search values (leave blank to skip a column). Example shown for each column:")
    
    # Take one example from the DataFrame for each column
    example_row = df.iloc[0]
    
    filters = {}
    for col in df.columns:
        example_value = example_row[col]
        value = input(f"{col} (example: {example_value}): ").strip()
        if value:  # only add filter if user entered something
            filters[col] = value
    
    result = df
    for key, value in filters.items():
        result = result[result[key] == value]
    
    if result.empty:
        print("\nNo matching games found.")
    else:
        print("\nMatching games:")
        print(result)

# Run the interactive search
interactive_search(parsed)


In [ ]:
import os
from SoccerNet.Downloader import SoccerNetDownloader
from SoccerNet.utils import getListGames
import pandas as pd

# 1️⃣ Your parsed DataFrame (from before)
# parsed = ...

# 2️⃣ Interactive search for a single game
def interactive_search(df):
    print("Enter search values (leave blank to skip a column). Example shown for each column:")
    example_row = df.iloc[0]
    filters = {}
    for col in df.columns:
        example_value = example_row[col]
        value = input(f"{col} (example: {example_value}): ").strip()
        if value:
            filters[col] = value

    result = df
    for key, value in filters.items():
        result = result[result[key] == value]

    if result.empty:
        print("\nNo matching games found.")
        return None
    else:
        print("\nMatching games:")
        print(result)
        return result.iloc[0]  # return first matching row

# 3️⃣ Set up the SoccerNet downloader
output_dir = os.path.join(os.getcwd(), "LaLiga_Videos")
password = "s0cc3rn3t"  # your NDA password
myDownloader = SoccerNetDownloader(LocalDirectory=output_dir)
myDownloader.password = password

# 4️⃣ Run search and download the corresponding game
game_row = interactive_search(parsed)
if game_row is not None:
    # Determine the SoccerNet game string (folder name)
    game_index = parsed.index.get_loc(game_row.name)  # get index of this game
    game_name = getListGames("all")[game_index]      # SoccerNet game name
    
    # Ask user which resolution to download
    res_choice = input("Which resolution do you want? Enter '720' or '224' (default 720): ").strip()
    if res_choice == "224":
        files_to_download = ["1_224p.mkv", "2_224p.mkv", "Labels.json"]
    else:
        files_to_download = ["1_720p.mkv", "2_720p.mkv", "Labels.json"]
    
    # Download the selected files
    myDownloader.downloadGameIndex(index=game_index, files=files_to_download)
    print(f"Downloaded {res_choice}p videos and labels for {game_name} to {output_dir}")


In [ ]:
import os
from SoccerNet.Downloader import SoccerNetDownloader
from SoccerNet.utils import getListGames
import pandas as pd

# 1️⃣ Your parsed DataFrame (from before)
# parsed = ...

# 2️⃣ Interactive search for a single game
def interactive_search(df):
    print("Enter search values (leave blank to skip a column). Example shown for each column:")
    example_row = df.iloc[0]
    filters = {}
    for col in df.columns:
        example_value = example_row[col]
        value = input(f"{col} (example: {example_value}): ").strip()
        if value:
            filters[col] = value

    result = df
    for key, value in filters.items():
        result = result[result[key] == value]

    if result.empty:
        print("\nNo matching games found.")
        return None
    else:
        print("\nMatching games:")
        print(result)
        return result

# 3️⃣ Set up the SoccerNet downloader
output_dir = os.path.join(os.getcwd(), "LaLiga_Videos")
password = "s0cc3rn3t"  # your NDA password
myDownloader = SoccerNetDownloader(LocalDirectory=output_dir)
myDownloader.password = password

# 4️⃣ Ask what the user wants to download
choice = input("Do you want to download a single game, a full session, or a league? (Enter 'game', 'session', 'league'): ").strip().lower()

# 5️⃣ Ask for resolution
res_choice = input("Which resolution? Enter '720' or '224' (default 720): ").strip()
if res_choice == "224":
    files_to_download = ["1_224p.mkv", "2_224p.mkv", "Labels.json"]
else:
    files_to_download = ["1_720p.mkv", "2_720p.mkv", "Labels.json"]

if choice == "game":
    game_row = interactive_search(parsed)
    if game_row is not None:
        game_index = parsed.index.get_loc(game_row.iloc[0].name)
        myDownloader.downloadGameIndex(index=game_index, files=files_to_download)
        print("Download completed for the selected game.")

elif choice == "session":
    session_input = input("Enter the season you want to download (example: 2014-2015): ").strip()
    session_games = parsed[parsed["season"] == session_input]
    count = len(session_games)
    print(f"There are {count} games in season {session_input}.")
    if count > 0:
        confirm = input(f"Do you want to download all {count} games? (y/n): ").strip().lower()
        if confirm == "y":
            for _, row in session_games.iterrows():
                game_name = row['league'] + '\\' + row['season'] + '\\' + f"{row['date']} - {row['time']} {row['home_team']} {row['home_score']} - {row['away_score']} {row['away_team']}"
                myDownloader.downloadGame(game_name, files=files_to_download)
            print(f"Download completed for all games in season {session_input}.")

elif choice == "league":
    league_input = input("Enter the league you want to download (example: england_epl): ").strip()
    league_games = parsed[parsed["league"] == league_input]
    count = len(league_games)
    print(f"There are {count} games in league {league_input}.")
    if count > 0:
        confirm = input(f"Do you want to download all {count} games? (y/n): ").strip().lower()
        if confirm == "y":
            for _, row in league_games.iterrows():
                game_name = row['league'] + '\\' + row['season'] + '\\' + f"{row['date']} - {row['time']} {row['home_team']} {row['home_score']} - {row['away_score']} {row['away_team']}"
                myDownloader.downloadGame(game_name, files=files_to_download)
            print(f"Download completed for all games in league {league_input}.")


else:
    print("Invalid choice. Please enter 'game', 'session', or 'league'.")


In [ ]:
import os
import pandas as pd
from SoccerNet.utils import getListGames
from SoccerNet.Downloader import SoccerNetDownloader

# --- 1. Parse games into DataFrame using SoccerNet game list ---
all_games = getListGames(split=["train","valid","test","challenge"])
df = pd.DataFrame(all_games, columns=["Games"])

pattern = r'^(?P<league>[^\\]+)\\(?P<season>[^\\]+)\\(?P<date>\d{4}-\d{2}-\d{2})\s*-\s*(?P<time>\d{2}-\d{2})\s+(?P<home_team>.+?)\s+(?P<home_score>\d+)\s*-\s*(?P<away_score>\d+)\s+(?P<away_team>.+)$'
parsed = df["Games"].str.extract(pattern)
parsed = parsed[["league","season","date","time","home_team","home_score","away_score","away_team"]]

# --- 2. Initialize SoccerNet Downloader ---
output_dir = os.path.join(os.getcwd(), "LaLiga_Videos")
password = "s0cc3rn3t"
downloader = SoccerNetDownloader(LocalDirectory=output_dir)
downloader.password = password

# --- 3. Ask user what to download ---
choice = input("Download a single game, a full season, or a league? (Enter 'game', 'season', 'league'): ").strip().lower()
res_choice = input("Select resolution (720 or 224, default 720): ").strip()
if res_choice == "224":
    files_to_download = ["1_224p.mkv", "2_224p.mkv", "Labels.json"]
else:
    files_to_download = ["1_720p.mkv", "2_720p.mkv", "Labels.json"]

# --- 4. Interactive search for single game ---
def interactive_search(df):
    print("\nEnter search values (leave blank to skip a column). Example shown for each column:")
    example_row = df.iloc[0]
    filters = {}
    for col in df.columns:
        example_value = example_row[col]
        value = input(f"{col} (example: {example_value}): ").strip()
        if value:
            filters[col] = value

    result = df
    for key, value in filters.items():
        result = result[result[key] == value]

    if result.empty:
        print("\nNo matching games found.")
        return None
    else:
        print("\nMatching games:")
        print(result)
        return result

# --- 5. Download logic ---
def download_games(filtered_df):
    count = len(filtered_df)
    print(f"\nTotal games to download: {count}")
    confirm = input("Do you want to download all these games? (y/n): ").strip().lower()
    if confirm != "y":
        print("Download cancelled.")
        return

    for _, row in filtered_df.iterrows():
        match_str = f"{row['league']}/{row['season']}/{row['date']} - {row['time']} {row['home_team']} {row['home_score']} - {row['away_score']} {row['away_team']}"
        matched_games = [g for g in all_games if g.replace("\\","/").endswith(match_str)]
        if matched_games:
            print(f"Downloading: {matched_games[0]}")
            downloader.downloadGame(matched_games[0], files=files_to_download)
        else:
            print(f"WARNING: Game not found in official list: {match_str}")

# --- 6. Execute based on user choice ---
if choice == "game":
    game_row = interactive_search(parsed)
    if game_row is not None:
        download_games(game_row)

elif choice == "season":
    season_input = input("Enter season (example: 2014-2015): ").strip()
    filtered = parsed[parsed["season"] == season_input]
    if len(filtered) == 0:
        print(f"No games found for season {season_input}.")
    else:
        download_games(filtered)

elif choice == "league":
    league_input = input("Enter league (example: england_epl): ").strip()
    league_games = parsed[parsed["league"] == league_input]
    if len(league_games) == 0:
        print(f"No games found for league {league_input}.")
    else:
        print(f"\nSeasons available in {league_input}: {league_games['season'].unique()}")
        season_input = input("Enter the season you want to download: ").strip()
        season_games = league_games[league_games["season"] == season_input]
        if len(season_games) == 0:
            print(f"No games found for season {season_input} in league {league_input}.")
        else:
            print(f"Number of games in {league_input} {season_input}: {len(season_games)}")
            download_games(season_games)

else:
    print("Invalid choice. Please enter 'game', 'season', or 'league'.")


In [ ]:
import os
import pandas as pd
from SoccerNet.utils import getListGames
from SoccerNet.Downloader import SoccerNetDownloader
from concurrent.futures import ThreadPoolExecutor

# --- 1. Parse games into DataFrame using SoccerNet game list ---
all_games = getListGames(split=["train","valid","test","challenge"])
df = pd.DataFrame(all_games, columns=["Games"])

pattern = r'^(?P<league>[^\\]+)\\(?P<season>[^\\]+)\\(?P<date>\d{4}-\d{2}-\d{2})\s*-\s*(?P<time>\d{2}-\d{2})\s+(?P<home_team>.+?)\s+(?P<home_score>\d+)\s*-\s*(?P<away_score>\d+)\s+(?P<away_team>.+)$'
parsed = df["Games"].str.extract(pattern)
parsed = parsed[["league","season","date","time","home_team","home_score","away_score","away_team"]]

# --- 2. Initialize SoccerNet Downloader ---
output_dir = os.path.join(os.getcwd(), "LaLiga_Videos")
password = "s0cc3rn3t"
downloader = SoccerNetDownloader(LocalDirectory=output_dir)
downloader.password = password

# --- 3. Ask user resolution ---
res_choice = input("Select resolution (720 or 224, default 720): ").strip()
if res_choice == "224":
    files_to_download = ["1_224p.mkv", "2_224p.mkv", "Labels.json"]
else:
    files_to_download = ["1_720p.mkv", "2_720p.mkv", "Labels.json"]

# --- 4. Interactive search for single game ---
def interactive_search(df):
    print("\nEnter search values (leave blank to skip a column). Example shown for each column:")
    example_row = df.iloc[0]
    filters = {}
    for col in df.columns:
        example_value = example_row[col]
        value = input(f"{col} (example: {example_value}): ").strip()
        if value:
            filters[col] = value

    result = df
    for key, value in filters.items():
        result = result[result[key] == value]

    if result.empty:
        print("\nNo matching games found.")
        return None
    else:
        print("\nMatching games:")
        print(result)
        return result

# --- 5. Multi-threaded download ---
def download_game_row(row):
    match_str = f"{row['league']}/{row['season']}/{row['date']} - {row['time']} {row['home_team']} {row['home_score']} - {row['away_score']} {row['away_team']}"
    matched_games = [g for g in all_games if g.replace("\\","/").endswith(match_str)]
    if matched_games:
        print(f"Downloading: {matched_games[0]}")
        downloader.downloadGame(matched_games[0], files=files_to_download)
    else:
        print(f"WARNING: Game not found in official list: {match_str}")

def download_games(filtered_df, max_workers=4):
    count = len(filtered_df)
    print(f"\nTotal games to download: {count}")
    confirm = input("Do you want to download all these games? (y/n): ").strip().lower()
    if confirm != "y":
        print("Download cancelled.")
        return
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        executor.map(download_game_row, [row for _, row in filtered_df.iterrows()])

# --- 6. Main menu ---
choice = input("Download a single game, a full season, or a league? (Enter 'game', 'season', 'league'): ").strip().lower()

if choice == "game":
    game_row = interactive_search(parsed)
    if game_row is not None:
        download_games(game_row)

elif choice == "season":
    print(f"Seasons available: {parsed['season'].unique()}")
    season_input = input("Enter the season you want to download: ").strip()
    season_games = parsed[parsed["season"] == season_input]
    if len(season_games) == 0:
        print(f"No games found for season {season_input}.")
    else:
        print(f"Number of games in season {season_input}: {len(season_games)}")
        download_games(season_games)

elif choice == "league":
    print(f"Leagues available: {parsed['league'].unique()}")
    league_input = input("Enter league: ").strip()
    league_games = parsed[parsed["league"] == league_input]
    if len(league_games) == 0:
        print(f"No games found for league {league_input}.")
    else:
        print(f"Seasons available in {league_input}: {league_games['season'].unique()}")
        season_input = input("Enter the season you want to download: ").strip()
        season_games = league_games[league_games["season"] == season_input]
        if len(season_games) == 0:
            print(f"No games found for season {season_input} in league {league_input}.")
        else:
            print(f"Number of games in {league_input} {season_input}: {len(season_games)}")
            download_games(season_games)

else:
    print("Invalid choice. Please enter 'game', 'season', or 'league'.")


In [ ]:
import os
import pandas as pd
from SoccerNet.utils import getListGames
from SoccerNet.Downloader import SoccerNetDownloader
from concurrent.futures import ThreadPoolExecutor

# --- 1. Parse games into DataFrame using SoccerNet game list ---
all_games = getListGames(split=["train","valid","test","challenge"])
df = pd.DataFrame(all_games, columns=["Games"])

pattern = r'^(?P<league>[^\\]+)\\(?P<season>[^\\]+)\\(?P<date>\d{4}-\d{2}-\d{2})\s*-\s*(?P<time>\d{2}-\d{2})\s+(?P<home_team>.+?)\s+(?P<home_score>\d+)\s*-\s*(?P<away_score>\d+)\s+(?P<away_team>.+)$'
parsed = df["Games"].str.extract(pattern)
parsed = parsed[["league","season","date","time","home_team","home_score","away_score","away_team"]]

# --- 2. Initialize SoccerNet Downloader ---
output_dir = os.path.join(os.getcwd(), "LaLiga_Videos")
password = "s0cc3rn3t"
downloader = SoccerNetDownloader(LocalDirectory=output_dir)
downloader.password = password

# --- 3. Ask user resolution ---
res_choice = input("Select resolution (720 or 224, default 720): ").strip()
if res_choice == "224":
    files_to_download = ["1_224p.mkv", "2_224p.mkv", "Labels.json"]
else:
    files_to_download = ["1_720p.mkv", "2_720p.mkv", "Labels.json"]

# --- 4. Interactive search for single game ---
def interactive_search(df):
    print("\nEnter search values (leave blank to skip a column). Example shown for each column:")
    example_row = df.iloc[0]
    filters = {}
    for col in df.columns:
        example_value = example_row[col]
        value = input(f"{col} (example: {example_value}): ").strip()
        if value:
            filters[col] = value

    result = df
    for key, value in filters.items():
        result = result[result[key] == value]

    if result.empty:
        print("\nNo matching games found.")
        return None
    else:
        print("\nMatching games:")
        print(result)
        return result

# --- 5. Multi-threaded download ---
def download_game_row(row):
    match_str = f"{row['league']}/{row['season']}/{row['date']} - {row['time']} {row['home_team']} {row['home_score']} - {row['away_score']} {row['away_team']}"
    matched_games = [g for g in all_games if g.replace("\\","/").endswith(match_str)]
    if matched_games:
        print(f"Downloading: {matched_games[0]}")
        downloader.downloadGame(matched_games[0], files=files_to_download)
    else:
        print(f"WARNING: Game not found in official list: {match_str}")

def download_games(filtered_df, max_workers=4):
    count = len(filtered_df)
    print(f"\nTotal games to download: {count}")
    confirm = input("Do you want to download all these games? (y/n): ").strip().lower()
    if confirm != "y":
        print("Download cancelled.")
        return
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        executor.map(download_game_row, [row for _, row in filtered_df.iterrows()])

# --- 6. Main menu ---
choice = input("Download a single game or a full league? (Enter 'game' or 'league'): ").strip().lower()

if choice == "game":
    game_row = interactive_search(parsed)
    if game_row is not None:
        download_games(game_row)

elif choice == "league":
    print(f"Leagues available: {parsed['league'].unique()}")
    league_input = input("Enter league: ").strip()
    league_games = parsed[parsed["league"] == league_input]
    if len(league_games) == 0:
        print(f"No games found for league {league_input}.")
    else:
        print(f"Seasons available in {league_input}: {league_games['season'].unique()}")
        season_input = input("Enter the season you want to download: ").strip()
        season_games = league_games[league_games["season"] == season_input]
        if len(season_games) == 0:
            print(f"No games found for season {season_input} in league {league_input}.")
        else:
            print(f"Number of games in {league_input} {season_input}: {len(season_games)}")
            download_games(season_games)

else:
    print("Invalid choice. Please enter 'game' or 'league'.")


In [ ]:
import os
import pandas as pd
from SoccerNet.utils import getListGames
from SoccerNet.Downloader import SoccerNetDownloader
from concurrent.futures import ThreadPoolExecutor

# --- 1. Parse games into DataFrame using SoccerNet game list ---
all_games = getListGames(split=["train","valid","test","challenge"])
df = pd.DataFrame(all_games, columns=["Games"])

pattern = r'^(?P<league>[^\\]+)\\(?P<season>[^\\]+)\\(?P<date>\d{4}-\d{2}-\d{2})\s*-\s*(?P<time>\d{2}-\d{2})\s+(?P<home_team>.+?)\s+(?P<home_score>\d+)\s*-\s*(?P<away_score>\d+)\s+(?P<away_team>.+)$'
parsed = df["Games"].str.extract(pattern)
parsed = parsed[["league","season","date","time","home_team","home_score","away_score","away_team"]]

# --- 2. Initialize SoccerNet Downloader ---
output_dir = os.path.join(os.getcwd(), "LaLiga_Videos")
password = "s0cc3rn3t"
downloader = SoccerNetDownloader(LocalDirectory=output_dir)
downloader.password = password

# --- 3. Ask user resolution ---
res_choice = input("Select resolution (720 or 224, default 720): ").strip()
if res_choice == "224":
    files_to_download = ["1_224p.mkv", "2_224p.mkv", "Labels.json"]
else:
    files_to_download = ["1_720p.mkv", "2_720p.mkv", "Labels.json"]

# --- 4. Interactive search for single game ---
def interactive_search(df):
    print("\nEnter search values (leave blank to skip a column). Example shown for each column:")
    example_row = df.iloc[0]
    filters = {}
    for col in df.columns:
        example_value = example_row[col]
        value = input(f"{col} (example: {example_value}): ").strip()
        if value:
            filters[col] = value

    result = df
    for key, value in filters.items():
        result = result[result[key] == value]

    if result.empty:
        print("\nNo matching games found.")
        return None
    else:
        print("\nMatching games:")
        print(result)

        # Save filtered result to CSV
        filtered_csv = os.path.join(os.getcwd(), "filtered_games.csv")
        result.to_csv(filtered_csv, index=False)
        print(f"\nFiltered search result saved to CSV: {filtered_csv}")
        return result

# --- 5. Multi-threaded download ---
def download_game_row(row):
    match_str = f"{row['league']}/{row['season']}/{row['date']} - {row['time']} {row['home_team']} {row['home_score']} - {row['away_score']} {row['away_team']}"
    matched_games = [g for g in all_games if g.replace("\\","/").endswith(match_str)]
    if matched_games:
        print(f"Downloading: {matched_games[0]}")
        downloader.downloadGame(matched_games[0], files=files_to_download)
    else:
        print(f"WARNING: Game not found in official list: {match_str}")

def download_games(filtered_df, max_workers=4):
    count = len(filtered_df)
    print(f"\nTotal games to download: {count}")
    confirm = input("Do you want to download all these games? (y/n): ").strip().lower()
    if confirm != "y":
        print("Download cancelled.")
        return
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        executor.map(download_game_row, [row for _, row in filtered_df.iterrows()])

# --- 6. Main menu ---
choice = input("Download a single game or a full league? (Enter 'game' or 'league'): ").strip().lower()

if choice == "game":
    game_row = interactive_search(parsed)
    if game_row is not None:
        download_games(game_row)

elif choice == "league":
    print(f"Leagues available: {parsed['league'].unique()}")
    league_input = input("Enter league: ").strip()
    league_games = parsed[parsed["league"] == league_input]
    if len(league_games) == 0:
        print(f"No games found for league {league_input}.")
    else:
        print(f"Seasons available in {league_input}: {league_games['season'].unique()}")
        season_input = input("Enter the season you want to download: ").strip()
        season_games = league_games[league_games["season"] == season_input]
        if len(season_games) == 0:
            print(f"No games found for season {season_input} in league {league_input}.")
        else:
            print(f"Number of games in {league_input} {season_input}: {len(season_games)}")

            # Save filtered league/season to CSV
            filtered_csv = os.path.join(os.getcwd(), f"{league_input}_{season_input}_games.csv")
            season_games.to_csv(filtered_csv, index=False)
            print(f"Filtered league/season games saved to CSV: {filtered_csv}")

            download_games(season_games)

else:
    print("Invalid choice. Please enter 'game' or 'league'.")


In [ ]:
import os
import pandas as pd
from SoccerNet.utils import getListGames
from SoccerNet.Downloader import SoccerNetDownloader
from concurrent.futures import ThreadPoolExecutor

# --- 1. Parse games into DataFrame using SoccerNet game list ---
all_games = getListGames(split=["train","valid","test","challenge"])
df = pd.DataFrame(all_games, columns=["Games"])

pattern = r'^(?P<league>[^\\]+)\\(?P<season>[^\\]+)\\(?P<date>\d{4}-\d{2}-\d{2})\s*-\s*(?P<time>\d{2}-\d{2})\s+(?P<home_team>.+?)\s+(?P<home_score>\d+)\s*-\s*(?P<away_score>\d+)\s+(?P<away_team>.+)$'
parsed = df["Games"].str.extract(pattern)
parsed = parsed[["league","season","date","time","home_team","home_score","away_score","away_team"]]

# --- 2. Initialize SoccerNet Downloader ---
output_dir = os.path.join(os.getcwd(), "LaLiga_Videos")
password = "s0cc3rn3t"
downloader = SoccerNetDownloader(LocalDirectory=output_dir)
downloader.password = password

# --- 3. Ask user resolution ---
res_choice = input("Select resolution (720 or 224, default 720): ").strip()
if res_choice == "224":
    files_to_download = ["1_224p.mkv", "2_224p.mkv", "Labels.json"]
else:
    files_to_download = ["1_720p.mkv", "2_720p.mkv", "Labels.json"]

# --- 4. Interactive search for single game ---
def interactive_search(df):
    print("\nSelect values from available options (leave blank to skip a column):")
    filters = {}
    
    for col in df.columns:
        if col in ["league", "season", "home_team", "away_team"]:
            unique_vals = df[col].unique()
            print(f"\nAvailable {col}s: {list(unique_vals)}")
        example_value = df[col].iloc[0]
        value = input(f"{col} (example: {example_value}): ").strip()
        if value:
            filters[col] = value

    result = df
    for key, value in filters.items():
        result = result[result[key] == value]

    if result.empty:
        print("\nNo matching games found.")
        return None
    else:
        print("\nMatching games:")
        print(result)

        # Save filtered result to CSV
        filtered_csv = os.path.join(os.getcwd(), "filtered_games.csv")
        result.to_csv(filtered_csv, index=False)
        print(f"\nFiltered search result saved to CSV: {filtered_csv}")
        return result

# --- 5. Multi-threaded download ---
def download_game_row(row):
    match_str = f"{row['league']}/{row['season']}/{row['date']} - {row['time']} {row['home_team']} {row['home_score']} - {row['away_score']} {row['away_team']}"
    matched_games = [g for g in all_games if g.replace("\\","/").endswith(match_str)]
    if matched_games:
        print(f"Downloading: {matched_games[0]}")
        downloader.downloadGame(matched_games[0], files=files_to_download)
    else:
        print(f"WARNING: Game not found in official list: {match_str}")

def download_games(filtered_df, max_workers=4):
    count = len(filtered_df)
    print(f"\nTotal games to download: {count}")
    confirm = input("Do you want to download all these games? (y/n): ").strip().lower()
    if confirm != "y":
        print("Download cancelled.")
        return
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        executor.map(download_game_row, [row for _, row in filtered_df.iterrows()])

# --- 6. Main menu ---
print("Download a single game")
game_row = interactive_search(parsed)
if game_row is not None:
    download_games(game_row)


In [ ]:
import os
import pandas as pd
from SoccerNet.utils import getListGames
from SoccerNet.Downloader import SoccerNetDownloader
from concurrent.futures import ThreadPoolExecutor

# --- 1. Parse games into DataFrame ---
all_games = getListGames(split=["train","valid","test","challenge"])
df = pd.DataFrame(all_games, columns=["Games"])

pattern = r'^(?P<league>[^\\]+)\\(?P<season>[^\\]+)\\(?P<date>\d{4}-\d{2}-\d{2})\s*-\s*(?P<time>\d{2}-\d{2})\s+(?P<home_team>.+?)\s+(?P<home_score>\d+)\s*-\s*(?P<away_score>\d+)\s+(?P<away_team>.+)$'
parsed = df["Games"].str.extract(pattern)
parsed = parsed[["league","season","date","time","home_team","home_score","away_score","away_team"]]

# --- 2. Initialize SoccerNet Downloader ---
output_dir = os.path.join(os.getcwd(), "LaLiga_Videos")
password = "s0cc3rn3t"
downloader = SoccerNetDownloader(LocalDirectory=output_dir)
downloader.password = password

# --- 3. Ask user resolution ---
res_choice = input("Select resolution (720 or 224, default 720): ").strip()
if res_choice == "224":
    files_to_download = ["1_224p.mkv", "2_224p.mkv", "Labels.json"]
else:
    files_to_download = ["1_720p.mkv", "2_720p.mkv", "Labels.json"]

# --- 4. Interactive search with dynamic filtering ---
def interactive_search(parsed):
    filtered = parsed.copy()
    filters = {}

    # League selection
    leagues = filtered['league'].unique()
    print(f"\nAvailable leagues: {list(leagues)}")
    league_input = input("Select league: ").strip()
    if league_input:
        filtered = filtered[filtered['league'] == league_input]
        filters['league'] = league_input

    # Season selection
    seasons = filtered['season'].unique()
    print(f"\nAvailable seasons for league '{league_input}': {list(seasons)}")
    season_input = input("Select season: ").strip()
    if season_input:
        filtered = filtered[filtered['season'] == season_input]
        filters['season'] = season_input

    # Home team selection
    home_teams = filtered['home_team'].unique()
    print(f"\nAvailable home teams for league '{league_input}', season '{season_input}': {list(home_teams)}")
    home_input = input("Select home team: ").strip()
    if home_input:
        filtered = filtered[filtered['home_team'] == home_input]
        filters['home_team'] = home_input

    # Away team selection
    away_teams = filtered['away_team'].unique()
    print(f"\nAvailable away teams for selected options: {list(away_teams)}")
    away_input = input("Select away team: ").strip()
    if away_input:
        filtered = filtered[filtered['away_team'] == away_input]
        filters['away_team'] = away_input

    # Optional: date selection
    dates = filtered['date'].unique()
    print(f"\nAvailable dates for selected options: {list(dates)}")
    date_input = input("Select date (or leave blank to skip): ").strip()
    if date_input:
        filtered = filtered[filtered['date'] == date_input]

    if filtered.empty:
        print("\nNo matching games found.")
        return None
    else:
        print("\nMatching games:")
        print(filtered)

        # Save filtered result to CSV
        filtered_csv = os.path.join(os.getcwd(), "filtered_games.csv")
        filtered.to_csv(filtered_csv, index=False)
        print(f"\nFiltered search result saved to CSV: {filtered_csv}")
        return filtered

# --- 5. Multi-threaded download ---
def download_game_row(row):
    match_str = f"{row['league']}/{row['season']}/{row['date']} - {row['time']} {row['home_team']} {row['home_score']} - {row['away_score']} {row['away_team']}"
    matched_games = [g for g in all_games if g.replace("\\","/").endswith(match_str)]
    if matched_games:
        print(f"Downloading: {matched_games[0]}")
        downloader.downloadGame(matched_games[0], files=files_to_download)
    else:
        print(f"WARNING: Game not found in official list: {match_str}")

def download_games(filtered_df, max_workers=4):
    count = len(filtered_df)
    print(f"\nTotal games to download: {count}")
    confirm = input("Do you want to download all these games? (y/n): ").strip().lower()
    if confirm != "y":
        print("Download cancelled.")
        return
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        executor.map(download_game_row, [row for _, row in filtered_df.iterrows()])

# --- 6. Main menu ---
print("Download a single game from SoccerNet")
game_row = interactive_search(parsed)
if game_row is not None:
    download_games(game_row)


In [ ]:
"""
SoccerNet Game Downloader
-------------------------
This script allows interactive search and download of SoccerNet games.
It supports filtering by league, season, home/away teams, and date.
The filtered search results are saved to CSV and can be downloaded in 720p or 224p resolution.

Dependencies:
    - pandas
    - SoccerNet
    - tqdm
"""

import os
import pandas as pd
from SoccerNet.utils import getListGames
from SoccerNet.Downloader import SoccerNetDownloader
from concurrent.futures import ThreadPoolExecutor

# ------------------------- CONFIGURATION -------------------------
CONFIG = {
    "output_dir": os.path.join(os.getcwd(), "SoccerNet_Videos"),  # Output directory
    "password": "s0cc3rn3t",                                       # NDA password for videos
    "max_workers": 4,                                               # Multi-threaded downloads
    "default_resolution": "224",                                    # Default download resolution
    "label_file": "Labels-v2.json"                                  # Default label file
}

# ------------------------- HELPER FUNCTIONS -------------------------
def fetch_games():
    """
    Fetches all available SoccerNet games and parses them into a structured DataFrame.
    Returns:
        parsed (pd.DataFrame): DataFrame with columns:
            league, season, date, time, home_team, home_score, away_score, away_team
        all_game: List of all games
    """
    all_games = getListGames(split=["train", "valid", "test", "challenge"])
    df = pd.DataFrame(all_games, columns=["Games"])
    
    pattern = (
        r'^(?P<league>[^\\]+)\\(?P<season>[^\\]+)\\'
        r'(?P<date>\d{4}-\d{2}-\d{2})\s*-\s*'
        r'(?P<time>\d{2}-\d{2})\s+'
        r'(?P<home_team>.+?)\s+(?P<home_score>\d+)\s*-\s*'
        r'(?P<away_score>\d+)\s+(?P<away_team>.+)$'
    )
    parsed = df["Games"].str.extract(pattern)
    parsed = parsed[["league","season","date","time","home_team","home_score","away_score","away_team"]] # column name for df
    return parsed, all_games

def select_resolution():
    """
    Prompt user to select download resolution.
    Returns:
        files_to_download (list[str]): List of files to download based on resolution.
    """
    choice = input(f"Select resolution (224 or 720, default {CONFIG['default_resolution']}): ").strip()
    
    if choice == "224":
        return ["1_224p.mkv", "2_224p.mkv", CONFIG["label_file"]]
    else:
        return ["1_720p.mkv", "2_720p.mkv", CONFIG["label_file"]]

def interactive_search(parsed_df):
    """
    Interactive search with dynamic filtering for game selection.
    Filters sequentially: league → season → home_team → away_team → date
    Returns:
        filtered_df (pd.DataFrame) or None
    """
    filtered = parsed_df.copy()

    # League
    leagues = filtered['league'].unique()
    print(f"\nAvailable leagues: {list(leagues)}")
    league_input = input("Select league (press enter if you want to consider all the leagues): ").strip()
    if league_input:
        filtered = filtered[filtered['league'] == league_input]

    # Season
    seasons = filtered['season'].unique()
    print(f"\nAvailable seasons for league '{league_input}': {list(seasons)}")
    season_input = input("Select season (press enter if you want to consider all the seasons): ").strip()
    if season_input:
        filtered = filtered[filtered['season'] == season_input]

    # Home Team
    home_teams = filtered['home_team'].unique()
    print(f"\nAvailable home teams: {list(home_teams)}")
    home_input = input("Select home team (press enter if you want to consider all the home teams): ").strip()
    if home_input:
        filtered = filtered[filtered['home_team'] == home_input]

    # Away Team
    away_teams = filtered['away_team'].unique()
    print(f"\nAvailable away teams: {list(away_teams)}")
    away_input = input("Select away team (press enter if you want to consider all the away teams): ").strip()
    if away_input:
        filtered = filtered[filtered['away_team'] == away_input]

    # Date (optional)
    dates = filtered['date'].unique()
    print(f"\nAvailable dates: {list(dates)}")
    date_input = input("Select date (press enter if you want to consider all the dates): ").strip()
    if date_input:
        filtered = filtered[filtered['date'] == date_input]

    if filtered.empty:
        print("\nNo matching games found.")
        return None

    print("\nMatching games:")
    print(filtered)

    # Save search result
    filtered_csv = os.path.join(CONFIG["output_dir"], "filtered_games.csv")
    os.makedirs(CONFIG["output_dir"], exist_ok=True)
    filtered.to_csv(filtered_csv, index=False)
    print(f"\nFiltered results saved to CSV: {filtered_csv}")

    return filtered

def download_game_row(row, all_games, downloader, files):
    """
    Download a game using SoccerNetDownloader.
    """
    match_str = f"{row['league']}/{row['season']}/{row['date']} - {row['time']} {row['home_team']} {row['home_score']} - {row['away_score']} {row['away_team']}"
    matched_games = [g for g in all_games if g.replace("\\","/").endswith(match_str)]
    if matched_games:
        print(f"Downloading: {matched_games[0]}")
        downloader.downloadGame(matched_games[0], files=files)
    else:
        print(f"WARNING: Game not found in official list: {match_str}")

def download_games(filtered_df, all_games, downloader, files, max_workers=4):
    """
    Multi-threaded download of all selected games.
    """
    count = len(filtered_df)
    print(f"\nTotal games to download: {count}")
    confirm = input(f"Ready to download {count} games? (y/n): ").strip().lower()
    if confirm != "y":
        print("Download cancelled.")
        return

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        executor.map(lambda row: download_game_row(row, all_games, downloader, files),
                     [row for _, row in filtered_df.iterrows()])

# ------------------------- MAIN EXECUTION -------------------------
def main():
    print("SoccerNet Interactive Game Downloader")

    # Fetch games
    parsed_df, all_games = fetch_games()

    # Initialize downloader
    downloader = SoccerNetDownloader(LocalDirectory=CONFIG["output_dir"])
    downloader.password = CONFIG["password"]

    # Select files resolution
    files_to_download = select_resolution()

    # Search and download game
    filtered_games = interactive_search(parsed_df)
    if filtered_games is not None:
        download_games(filtered_games, all_games, downloader, files_to_download, CONFIG["max_workers"])

if __name__ == "__main__":
    main()


In [6]:
"""
SoccerNet Game Downloader
-------------------------
This script allows interactive search and download of SoccerNet games.
It supports filtering by league, season, home/away teams, and date.
The filtered search results are saved to CSV and can be downloaded 
in 720p or 224p resolution.

Dependencies:
    - pandas
    - SoccerNet
    - tqdm
"""

import os
import pandas as pd
from SoccerNet.utils import getListGames
from SoccerNet.Downloader import SoccerNetDownloader
from concurrent.futures import ThreadPoolExecutor

# =====================================================================
# CONFIGURATION
# =====================================================================
try:
    BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))  
except NameError:
    # Fallback for Jupyter/IPython (no __file__)
    BASE_DIR = os.path.dirname(os.getcwd())

OUTPUT_DIR = os.path.join(BASE_DIR, "dataset", "full_length_soccer_match_and_annotation")

# Make sure the directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

CONFIG = {
    "output_dir": OUTPUT_DIR,  # Output directory for downloads
    "password": "s0cc3rn3t",                                       # NDA password for videos
    "max_workers": 4,                                              # Threads for parallel downloads
    "default_resolution": "720",                                   # Default resolution (224 or 720)
    "label_file": "Labels-v2.json"                                 # Label file
}

# =====================================================================
# HELPER FUNCTIONS
# =====================================================================
def fetch_games():
    """
    Fetch all SoccerNet games and parse them into a structured DataFrame.
    
    Returns:
        parsed_df (pd.DataFrame): Columns = league, season, date, time, home_team, home_score, away_score, away_team
        all_games (list[str]): Raw list of all games (paths)
    """
    all_games = getListGames(split=["train", "valid", "test", "challenge"])
    df = pd.DataFrame(all_games, columns=["Games"])
    
    pattern = (
        r'^(?P<league>[^\\]+)\\(?P<season>[^\\]+)\\'
        r'(?P<date>\d{4}-\d{2}-\d{2})\s*-\s*'
        r'(?P<time>\d{2}-\d{2})\s+'
        r'(?P<home_team>.+?)\s+(?P<home_score>\d+)\s*-\s*'
        r'(?P<away_score>\d+)\s+(?P<away_team>.+)$'
    )
    
    parsed_df = df["Games"].str.extract(pattern)
    parsed_df = parsed_df[["league", "season", "date", "time", "home_team", "home_score", "away_score", "away_team"]]
    return parsed_df, all_games


def select_resolution():
    """
    Prompt user to select download resolution.
    
    Returns:
        list[str]: Files to download (video + labels).
    """
    choice = input(f"Select resolution (224 or 720, default {CONFIG['default_resolution']}): ").strip()
    if choice == "224":
        return ["1_224p.mkv", "2_224p.mkv", CONFIG["label_file"]]
    return ["1_720p.mkv", "2_720p.mkv", CONFIG["label_file"]]


def interactive_search(parsed_df):
    """
    Interactive guided search for filtering games.
    """
    filtered = parsed_df.copy()

    # League
    leagues = filtered['league'].unique()
    print(f"\nAvailable leagues: {list(leagues)}")
    league_input = input("Select league (Enter to skip): ").strip()
    if league_input:
        filtered = filtered[filtered['league'] == league_input]

    # Season
    seasons = filtered['season'].unique()
    print(f"\nAvailable seasons: {list(seasons)}")
    season_input = input("Select season (Enter to skip): ").strip()
    if season_input:
        filtered = filtered[filtered['season'] == season_input]

    # Home Team
    home_teams = filtered['home_team'].unique()
    print(f"\nAvailable home teams: {list(home_teams)}")
    home_input = input("Select home team (Enter to skip): ").strip()
    if home_input:
        filtered = filtered[filtered['home_team'] == home_input]

    # Away Team
    away_teams = filtered['away_team'].unique()
    print(f"\nAvailable away teams: {list(away_teams)}")
    away_input = input("Select away team (Enter to skip): ").strip()
    if away_input:
        filtered = filtered[filtered['away_team'] == away_input]

    # Date
    dates = filtered['date'].unique()
    print(f"\nAvailable dates: {list(dates)}")
    date_input = input("Select date (Enter to skip): ").strip()
    if date_input:
        filtered = filtered[filtered['date'] == date_input]

    if filtered.empty:
        print("\n❌ No matching games found.")
        return None

    print("\n✅ Matching games:")
    print(filtered)

    # Save search result to CSV
    os.makedirs(CONFIG["output_dir"], exist_ok=True)
    filtered_csv = os.path.join(CONFIG["output_dir"], "filtered_games.csv")
    filtered.to_csv(filtered_csv, index=False)
    print(f"\n📂 Filtered results saved to: {filtered_csv}")
    print("✅ CSV export completed successfully.")

    return filtered

def download_game_row(row, all_games, downloader, files):
    """
    Download a single game from SoccerNet.
    
    Args:
        row (pd.Series): Game row containing metadata.
        all_games (list[str]): List of all game paths.
        downloader (SoccerNetDownloader): Downloader object.
        files (list[str]): Files to download.
    """
    match_str = (
        f"{row['league']}/{row['season']}/{row['date']} - {row['time']} "
        f"{row['home_team']} {row['home_score']} - {row['away_score']} {row['away_team']}"
    )
    matched_games = [g for g in all_games if g.replace("\\", "/").endswith(match_str)]
    
    if matched_games:
        print(f"⬇️ Downloading: {matched_games[0]}")
        downloader.downloadGame(matched_games[0], files=files)
    else:
        print(f"⚠️ WARNING: Game not found in list → {match_str}")


def download_games(filtered_df, all_games, downloader, files, max_workers=4):
    """
    Multi-threaded download of selected games.
    """
    count = len(filtered_df)
    print(f"\nTotal games to download: {count}")
    confirm = input(f"Ready to download {count} games? (y/n): ").strip().lower()
    if confirm != "y":
        print("❌ Download cancelled.")
        return

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        executor.map(lambda row: download_game_row(row, all_games, downloader, files),
                     [row for _, row in filtered_df.iterrows()])

    print("\n✅ All selected matches downloaded successfully.")


# =====================================================================
# MAIN EXECUTION
# =====================================================================
def main():
    print("\n⚽ SoccerNet Interactive Game Downloader ⚽")

    # Fetch all games
    parsed_df, all_games = fetch_games()

    # Setup downloader
    downloader = SoccerNetDownloader(LocalDirectory=CONFIG["output_dir"])
    downloader.password = CONFIG["password"]

    # Select resolution (files to download)
    files_to_download = select_resolution()

    # Run interactive search + download
    filtered_games = interactive_search(parsed_df)
    if filtered_games is not None:
        download_games(filtered_games, all_games, downloader, files_to_download, CONFIG["max_workers"])


if __name__ == "__main__":
    main()


C:\Users\Yasin\Desktop\Publication\SBD_Soccer\src\dataset\full_length_soccer_match_and_annotation

⚽ SoccerNet Interactive Game Downloader ⚽


KeyboardInterrupt: Interrupted by user

In [16]:
"""
SoccerNet Game Downloader
-------------------------
This script allows interactive search and download of SoccerNet games.
It supports filtering by league, season, home/away teams, and date.
The filtered search results are saved to CSV and can be downloaded 
in 720p or 224p resolution.

Dependencies:
    - pandas
    - SoccerNet
    - tqdm
"""

import os
import pandas as pd
from SoccerNet.utils import getListGames
from SoccerNet.Downloader import SoccerNetDownloader
from concurrent.futures import ThreadPoolExecutor

# =====================================================================
# CONFIGURATION
# =====================================================================
try:
    # When running as a script
    BASE_DIR = os.path.abspath(os.path.join(os.path.dirname(__file__), "..", ".."))
except NameError:
    # Fallback for Jupyter/IPython
    BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))

# Output directory relative to project root
OUTPUT_DIR = os.path.join(BASE_DIR, "dataset", "full_length_soccer_match_and_annotation")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# =====================================================================
# HELPER FUNCTIONS
# =====================================================================
def fetch_games():
    """Fetch all SoccerNet games and parse them into a structured DataFrame."""
    all_games = getListGames(split=["train", "valid", "test", "challenge"])
    df = pd.DataFrame(all_games, columns=["Games"])
    
    pattern = (
        r'^(?P<league>[^\\]+)\\(?P<season>[^\\]+)\\'
        r'(?P<date>\d{4}-\d{2}-\d{2})\s*-\s*'
        r'(?P<time>\d{2}-\d{2})\s+'
        r'(?P<home_team>.+?)\s+(?P<home_score>\d+)\s*-\s*'
        r'(?P<away_score>\d+)\s+(?P<away_team>.+)$'
    )
    
    parsed_df = df["Games"].str.extract(pattern)
    parsed_df = parsed_df[["league", "season", "date", "time", "home_team", "home_score", "away_score", "away_team"]]
    return parsed_df, all_games


def select_resolution():
    """Prompt user to select download resolution."""
    choice = input(f"Select resolution (224 or 720, default {CONFIG['default_resolution']}): ").strip()
    if choice == "224":
        return ["1_224p.mkv", "2_224p.mkv", CONFIG["label_file"]]
    return ["1_720p.mkv", "2_720p.mkv", CONFIG["label_file"]]


def interactive_search(parsed_df):
    """Interactive guided search for filtering games (supports multiple selections)."""
    filtered = parsed_df.copy()

    # Helper to handle multi-selection
    def multi_select(column_name, display_name):
        unique_values = filtered[column_name].unique()
        print(f"\nAvailable {display_name}: {list(unique_values)}")
        user_input = input(f"Select {display_name} (comma-separated, Enter to skip): ").strip()
        if user_input:
            choices = [x.strip() for x in user_input.split(",")]
            return filtered[filtered[column_name].isin(choices)]
        return filtered

    filtered = multi_select('league', 'leagues')
    filtered = multi_select('season', 'seasons')
    filtered = multi_select('home_team', 'home teams')
    filtered = multi_select('away_team', 'away teams')
    filtered = multi_select('date', 'dates')

    if filtered.empty:
        print("\n❌ No matching games found.")
        return None

    print("\n✅ Matching games:")
    print(filtered)

    # Save search result to CSV
    os.makedirs(CONFIG["output_dir"], exist_ok=True)
    filtered_csv = os.path.join(CONFIG["output_dir"], "filtered_games.csv")
    filtered.to_csv(filtered_csv, index=False)
    print(f"\n📂 Filtered results saved to: {filtered_csv}")
    print("✅ CSV export completed successfully.")

    return filtered


def download_game_row(row, all_games, downloader, files):
    """Download a single game from SoccerNet."""
    match_str = (
        f"{row['league']}/{row['season']}/{row['date']} - {row['time']} "
        f"{row['home_team']} {row['home_score']} - {row['away_score']} {row['away_team']}"
    )
    matched_games = [g for g in all_games if g.replace("\\", "/").endswith(match_str)]
    
    if matched_games:
        print(f"⬇️ Downloading: {matched_games[0]}")
        downloader.downloadGame(matched_games[0], files=files)
    else:
        print(f"⚠️ WARNING: Game not found → {match_str}")


def download_games(filtered_df, all_games, downloader, files, max_workers=4):
    """Multi-threaded download of selected games."""
    count = len(filtered_df)
    print(f"\nTotal games to download: {count}")
    confirm = input(f"Ready to download {count} games? (y/n): ").strip().lower()
    if confirm != "y":
        print("❌ Download cancelled.")
        return

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        executor.map(lambda row: download_game_row(row, all_games, downloader, files),
                     [row for _, row in filtered_df.iterrows()])

    print("\n✅ All selected matches downloaded successfully.")


# =====================================================================
# MAIN EXECUTION
# =====================================================================
def main():
    print("\n⚽ SoccerNet Interactive Game Downloader ⚽")

    # Fetch all games
    parsed_df, all_games = fetch_games()

    # Setup downloader
    downloader = SoccerNetDownloader(LocalDirectory=CONFIG["output_dir"])
    downloader.password = CONFIG["password"]

    # Select resolution (files to download)
    files_to_download = select_resolution()

    # Run interactive search + download
    filtered_games = interactive_search(parsed_df)
    if filtered_games is not None:
        download_games(filtered_games, all_games, downloader, files_to_download, CONFIG["max_workers"])


if __name__ == "__main__":
    main()

    


⚽ SoccerNet Interactive Game Downloader ⚽
Select resolution (224 or 720, default 720): 224

Available leagues: ['england_epl', 'europe_uefa-champions-league', 'france_ligue-1', 'germany_bundesliga', 'italy_serie-a', 'spain_laliga']
Select leagues (comma-separated, Enter to skip): 

Available seasons: ['2014-2015', '2015-2016', '2016-2017', '2019-2020']
Select seasons (comma-separated, Enter to skip): 2019-2020

Available home teams: ['Real Madrid', 'Barcelona', 'Osasuna', 'Valencia', 'Real Sociedad', 'Celta Vigo']
Select home teams (comma-separated, Enter to skip): Celta Vigo

Available away teams: ['Real Madrid']
Select away teams (comma-separated, Enter to skip): 

Available dates: ['2019-08-17']
Select dates (comma-separated, Enter to skip): 

✅ Matching games:
           league     season        date   time   home_team home_score  \
549  spain_laliga  2019-2020  2019-08-17  18-00  Celta Vigo          1   

    away_score    away_team  
549          3  Real Madrid  

📂 Filtered res

HTTP Error 404: Not Found

✅ All selected matches downloaded successfully.


In [18]:
"""
SoccerNet Game Downloader (Flat Structure)
------------------------------------------
Downloads SoccerNet matches into a flat folder.
Supports interactive filtering and multiple selections.
"""

import os
import pandas as pd
import shutil
from SoccerNet.utils import getListGames
from SoccerNet.Downloader import SoccerNetDownloader
from concurrent.futures import ThreadPoolExecutor

# =====================================================================
# CONFIGURATION
# =====================================================================
try:
    BASE_DIR = os.path.abspath(os.path.join(os.path.dirname(__file__), "..", ".."))
except NameError:
    BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))

OUTPUT_DIR = os.path.join(BASE_DIR, "dataset", "full_length_soccer_match_and_annotation")
os.makedirs(OUTPUT_DIR, exist_ok=True)

CONFIG = {
    "output_dir": OUTPUT_DIR,
    "password": "s0cc3rn3t",
    "max_workers": 4,
    "default_resolution": "720",
    "label_file": "Labels-v2.json"
}

# =====================================================================
# HELPER FUNCTIONS
# =====================================================================
def fetch_games():
    all_games = getListGames(split=["train", "valid", "test", "challenge"])
    df = pd.DataFrame(all_games, columns=["Games"])

    pattern = (
        r'^(?P<league>[^\\]+)\\(?P<season>[^\\]+)\\'
        r'(?P<date>\d{4}-\d{2}-\d{2})\s*-\s*'
        r'(?P<time>\d{2}-\d{2})\s+'
        r'(?P<home_team>.+?)\s+(?P<home_score>\d+)\s*-\s*'
        r'(?P<away_score>\d+)\s+(?P<away_team>.+)$'
    )

    parsed_df = df["Games"].str.extract(pattern)
    parsed_df = parsed_df[["league", "season", "date", "time", "home_team", "home_score", "away_score", "away_team"]]
    return parsed_df, all_games


def select_resolution():
    choice = input(f"Select resolution (224 or 720, default {CONFIG['default_resolution']}): ").strip()
    if choice == "224":
        return ["1_224p.mkv", "2_224p.mkv", CONFIG["label_file"]]
    return ["1_720p.mkv", "2_720p.mkv", CONFIG["label_file"]]


def interactive_search(parsed_df):
    filtered = parsed_df.copy()

    def multi_select(column_name, display_name):
        unique_values = filtered[column_name].unique()
        print(f"\nAvailable {display_name}: {list(unique_values)}")
        user_input = input(f"Select {display_name} (comma-separated, Enter to skip): ").strip()
        if user_input:
            choices = [x.strip() for x in user_input.split(",")]
            return filtered[filtered[column_name].isin(choices)]
        return filtered

    filtered = multi_select('league', 'leagues')
    filtered = multi_select('season', 'seasons')
    filtered = multi_select('home_team', 'home teams')
    filtered = multi_select('away_team', 'away teams')
    filtered = multi_select('date', 'dates')

    if filtered.empty:
        print("\n❌ No matching games found.")
        return None

    print("\n✅ Matching games:")
    print(filtered)

    filtered_csv = os.path.join(CONFIG["output_dir"], "filtered_games.csv")
    filtered.to_csv(filtered_csv, index=False)
    print(f"\n📂 Filtered results saved to: {filtered_csv}")

    return filtered


def download_game_row(row, all_games, downloader, files):
    match_str = (
        f"{row['league']}/{row['season']}/{row['date']} - {row['time']} "
        f"{row['home_team']} {row['home_score']} - {row['away_score']} {row['away_team']}"
    )
    matched_games = [g for g in all_games if g.replace("\\", "/").endswith(match_str)]

    if not matched_games:
        print(f"⚠️ WARNING: Game not found → {match_str}")
        return

    print(f"⬇️ Downloading: {matched_games[0]}")

    # Download to default nested structure
    downloader.downloadGame(matched_games[0], files=files)

    # Move to flat folder
    nested_dir = os.path.join(CONFIG["output_dir"], matched_games[0].replace("/", os.sep))
    flat_name = f"{row['date']} - {row['time']} {row['home_team']} {row['home_score']} - {row['away_score']} {row['away_team']}"
    target_dir = os.path.join(CONFIG["output_dir"], flat_name)
    os.makedirs(target_dir, exist_ok=True)

    for f in files:
        src = os.path.join(nested_dir, f)
        if os.path.exists(src):
            shutil.move(src, os.path.join(target_dir, f))

    # Clean up empty nested folders
    try:
        shutil.rmtree(os.path.join(CONFIG["output_dir"], row['league']))
    except Exception:
        pass

    print(f"✅ Saved match in flat folder: {target_dir}")


def download_games(filtered_df, all_games, downloader, files, max_workers=4):
    count = len(filtered_df)
    print(f"\nTotal games to download: {count}")
    confirm = input(f"Ready to download {count} games? (y/n): ").strip().lower()
    if confirm != "y":
        print("❌ Download cancelled.")
        return

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        executor.map(lambda row: download_game_row(row, all_games, downloader, files),
                     [row for _, row in filtered_df.iterrows()])

    print("\n✅ All selected matches downloaded successfully.")


def main():
    print("\n⚽ SoccerNet Interactive Game Downloader ⚽")

    parsed_df, all_games = fetch_games()

    downloader = SoccerNetDownloader(LocalDirectory=CONFIG["output_dir"])
    downloader.password = CONFIG["password"]

    files_to_download = select_resolution()

    filtered_games = interactive_search(parsed_df)
    if filtered_games is not None:
        download_games(filtered_games, all_games, downloader, files_to_download, CONFIG["max_workers"])


if __name__ == "__main__":
    main()



⚽ SoccerNet Interactive Game Downloader ⚽
Select resolution (224 or 720, default 720): 224

Available leagues: ['england_epl', 'europe_uefa-champions-league', 'france_ligue-1', 'germany_bundesliga', 'italy_serie-a', 'spain_laliga']
Select leagues (comma-separated, Enter to skip): england_epl,spain_laliga

Available seasons: ['2014-2015', '2015-2016', '2016-2017', '2019-2020']
Select seasons (comma-separated, Enter to skip): 2014-2015,2015-2016

Available home teams: ['Chelsea', 'Crystal Palace', 'Swansea', 'Southampton', 'Manchester City', 'Everton', 'Liverpool', 'Manchester United', 'Tottenham', 'Sunderland', 'Barcelona', 'Elche', 'Real Madrid', 'Atl. Madrid', 'Espanyol', 'Ath Bilbao', 'Eibar', 'Betis', 'Granada CF', 'Malaga', 'Las Palmas', 'Getafe', 'Real Sociedad', 'Burnley', 'Leicester', 'West Ham', 'Arsenal', 'Bournemouth', 'Sevilla', 'Celta Vigo', 'Villarreal', 'Valencia', 'West Brom', 'Newcastle Utd', 'Stoke City', 'Watford', 'Norwich', 'Cordoba', 'Levante', 'Dep. La Coruna']
S

  0%|          | 0.00/41.2k [00:00<?, ?iB/s]

✅ Saved match in flat folder: C:\Users\Yasin\Desktop\Publication\SBD_Soccer\dataset\full_length_soccer_match_and_annotation\2016-04-02 - 21-30 Barcelona 1 - 2 Real Madrid


✅ Saved match in flat folder: C:\Users\Yasin\Desktop\Publication\SBD_Soccer\dataset\full_length_soccer_match_and_annotation\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley

✅ All selected matches downloaded successfully.
